In [ ]:
# create_noisy_dataset_combo.py
# -*- coding: utf-8 -*-
"""
5 gürültü sınıfı (gaussian, poisson, salt, speckle, perlin) için:
- Sınıf başına 2000 görüntü üretir (varsayılan)
- 1800 tek gürültü, 200 kombinasyon (dominant gürültü = klasör)
- Gürültü maskeleme: tam sayfa, yarım (dikey/yatay), şerit/çizgi
- Kombinasyonda maskeler çakışmaz, dominantın alanı daha büyük ayarlanır

Örnek:
python create_noisy_dataset_combo.py --out_dir ./noisy_dataset \
  --per_class 2000 --single_per_class 1800 --combo_per_class 200 --resize 128 --seed 42
"""

import os
import math
import argparse
import random
from pathlib import Path
from typing import List, Tuple, Optional, Dict

import numpy as np
from PIL import Image, ImageOps
import torch
from torchvision import datasets, transforms

# ----------------------------
# Yardımcılar
# ----------------------------
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def load_source_images_from_dir(source_dir: Path) -> List[Image.Image]:
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tif", ".tiff"}
    files = [p for p in source_dir.rglob("*") if p.suffix.lower() in exts]
    if not files:
        raise FileNotFoundError(f"Kaynak görsel bulunamadı: {source_dir}")
    imgs = []
    for p in files:
        try:
            imgs.append(Image.open(p).convert("RGB"))
        except Exception:
            pass
    if not imgs:
        raise RuntimeError("Geçerli görüntü açılamadı.")
    return imgs

def download_cifar10(root: Path, resize: int = 128) -> List[Image.Image]:
    ensure_dir(root)
    if resize and resize > 0:
        tfm = transforms.Compose([
            transforms.Resize((resize, resize), interpolation=Image.BICUBIC),
            transforms.ToTensor()
        ])
    else:
        tfm = transforms.ToTensor()
    ds = datasets.CIFAR10(root=str(root), train=True, download=True, transform=tfm)
    imgs: List[Image.Image] = []
    for i in range(len(ds)):
        t, _ = ds[i]  # C,H,W tensor [0,1]
        arr = (t.numpy().transpose(1,2,0) * 255.0).clip(0,255).astype(np.uint8)
        imgs.append(Image.fromarray(arr, mode="RGB"))
    return imgs  # ~50k

def to_np(img: Image.Image) -> np.ndarray:
    if img.mode != "RGB":
        img = img.convert("RGB")
    return np.asarray(img, dtype=np.float32) / 255.0

def to_img(arr: np.ndarray) -> Image.Image:
    arr = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
    return Image.fromarray(arr, mode="RGB")

def rand_float(a: float, b: float) -> float:
    return a + (b - a) * random.random()

# ----------------------------
# Gürültüler
# ----------------------------
def add_gaussian(img: Image.Image) -> Image.Image:
    x = to_np(img)
    sigma = rand_float(0.01, 0.08)
    n = np.random.normal(0.0, sigma, x.shape).astype(np.float32)
    y = np.clip(x + n, 0, 1)
    return to_img(y)

def add_poisson(img: Image.Image) -> Image.Image:
    x = to_np(img)
    vals = 2 ** np.ceil(rand_float(3, 6))
    y = np.random.poisson(x * vals) / vals
    y = np.clip(y, 0, 1).astype(np.float32)
    return to_img(y)

def add_salt(img: Image.Image) -> Image.Image:
    x = to_np(img)
    prob = rand_float(0.003, 0.02)
    rnd = np.random.rand(*x.shape[:2])
    salt = rnd < (prob / 2.0)
    pepper = rnd > (1 - prob / 2.0)
    y = x.copy()
    y[salt] = 1.0
    y[pepper] = 0.0
    return to_img(y)

def add_speckle(img: Image.Image) -> Image.Image:
    x = to_np(img)
    sigma = rand_float(0.02, 0.15)
    n = np.random.normal(0.0, sigma, x.shape).astype(np.float32)
    y = np.clip(x + x * n, 0, 1)
    return to_img(y)

# Basit Perlin benzeri aydınlatma
def _fade(t: np.ndarray) -> np.ndarray:
    return 6*t**5 - 15*t**4 + 10*t**3

def _lerp(a: np.ndarray, b: np.ndarray, t: np.ndarray) -> np.ndarray:
    return a + t * (b - a)

def _perlin_2d(h, w, scale):
    gx = int(math.ceil(w / scale)) + 2
    gy = int(math.ceil(h / scale)) + 2
    theta = np.random.rand(gy, gx) * 2 * np.pi
    grads = np.dstack((np.cos(theta), np.sin(theta)))
    yy, xx = np.mgrid[0:h, 0:w]
    x = xx / scale; y = yy / scale
    x0 = x.astype(int); x1 = x0 + 1
    y0 = y.astype(int); y1 = y0 + 1

    def grad(ix, iy): return grads[iy, ix]
    def dot_grid(ix, iy, x, y):
        g = grad(ix, iy); dx = x - ix; dy = y - iy
        return dx * g[...,0] + dy * g[...,1]

    n00 = dot_grid(x0, y0, x, y)
    n10 = dot_grid(x1, y0, x, y)
    n01 = dot_grid(x0, y1, x, y)
    n11 = dot_grid(x1, y1, x, y)
    u = _fade(x - x0); v = _fade(y - y0)
    nx0 = _lerp(n00, n10, u)
    nx1 = _lerp(n01, n11, u)
    nxy = _lerp(nx0, nx1, v)
    nmin, nmax = nxy.min(), nxy.max()
    if nmax - nmin < 1e-6: return np.zeros_like(nxy)
    return (nxy - nmin) / (nmax - nmin)

def add_perlin(img: Image.Image) -> Image.Image:
    x = to_np(img)
    h, w = x.shape[:2]
    octaves = random.choice([1,2,3])
    scale = rand_float(48, 128)
    strength = rand_float(0.1, 0.35)
    noise = np.zeros((h, w), dtype=np.float32)
    amp = 1.0; total = 0.0
    for o in range(octaves):
        noise += _perlin_2d(h, w, scale/(2**o)) * amp
        total += amp
        amp *= 0.5
    noise /= max(total, 1e-6)
    noise = (noise - 0.5) * 2.0
    mask = 1.0 + strength * noise[..., None]
    y = np.clip(x * mask, 0, 1)
    return to_img(y)

NOISE_FUNCS = {
    "gaussian": add_gaussian,
    "poisson": add_poisson,
    "salt": add_salt,
    "speckle": add_speckle,
    "perlin": add_perlin,
}

# ----------------------------
# Maske üretimi (overlap yok)
# ----------------------------
def mask_full(h: int, w: int) -> np.ndarray:
    return np.ones((h, w), dtype=bool)

def mask_half(h: int, w: int) -> np.ndarray:
    if random.random() < 0.5:  # dikey yarım
        x0 = 0 if random.random() < 0.5 else w//2
        x1 = w//2 if x0 == 0 else w
        m = np.zeros((h, w), dtype=bool)
        m[:, x0:x1] = True
    else:  # yatay yarım
        y0 = 0 if random.random() < 0.5 else h//2
        y1 = h//2 if y0 == 0 else h
        m = np.zeros((h, w), dtype=bool)
        m[y0:y1, :] = True
    return m

def mask_strip(h: int, w: int) -> np.ndarray:
    # çizgi/şerit: yatay ya da dikey, genişlik rastgele
    m = np.zeros((h, w), dtype=bool)
    if random.random() < 0.5:
        # dikey şerit
        width = max(2, int(rand_float(0.05, 0.2) * w))
        x0 = random.randint(0, w - width)
        m[:, x0:x0+width] = True
    else:
        # yatay şerit
        height = max(2, int(rand_float(0.05, 0.2) * h))
        y0 = random.randint(0, h - height)
        m[y0:y0+height, :] = True
    return m

MASK_FUNCS = [mask_full, mask_half, mask_strip]

def make_mask(h: int, w: int, target_area_ratio: float, max_trials: int = 50) -> np.ndarray:
    """
    İstenen alan oranına en yakın maskeyi basitçe dener. (full/half/strip)
    """
    best = None; best_diff = 1e9
    for _ in range(max_trials):
        m = random.choice(MASK_FUNCS)(h, w)
        ar = m.mean()
        diff = abs(ar - target_area_ratio)
        if diff < best_diff:
            best, best_diff = m, diff
            if best_diff < 0.02:  # yeterince yakın
                break
    return best

def apply_noise_with_mask(img: Image.Image, mask: np.ndarray, noise_fn) -> Image.Image:
    base = to_np(img)
    noisy_img = noise_fn(img)
    noisy = to_np(noisy_img)
    out = base.copy()
    out[mask] = noisy[mask]
    return to_img(out)

# ----------------------------
# Üretim mantığı
# ----------------------------
def generate_single(img: Image.Image, noise_name: str) -> Tuple[Image.Image, Dict]:
    h, w = img.size[1], img.size[0]
    m = random.choice(MASK_FUNCS)(h, w)
    out = apply_noise_with_mask(img, m, NOISE_FUNCS[noise_name])
    meta = {"type": "single", "primary": noise_name, "mask_area": float(m.mean())}
    return out, meta

def generate_combo(img: Image.Image, primary: str, partner: str) -> Tuple[Image.Image, Dict]:
    """
    İki gürültü uygula; maskeler çakışmasın. primary alanı partner'dan büyük olsun.
    """
    x = to_np(img); h, w = x.shape[:2]
    # alan oranları: primary %40–80, partner %10–40, toplam <= 90% (biraz boşluk kalsın)
    primary_ar = rand_float(0.45, 0.7)
    partner_ar = rand_float(0.1, min(0.35, 0.9 - primary_ar - 0.05))
    # maske üret
    m1 = make_mask(h, w, primary_ar)
    # partner maskesi m1 ile overlap etmeyecek şekilde dene
    trials = 0
    while True:
        m2 = make_mask(h, w, partner_ar)
        if not (m1 & m2).any():
            break
        trials += 1
        if trials > 200:
            # vazgeç → m2'yi m1 dışına kırp
            m2 = np.logical_and(m2, ~m1)
            break

    # uygula
    tmp = apply_noise_with_mask(img, m1, NOISE_FUNCS[primary])
    out = apply_noise_with_mask(tmp, m2, NOISE_FUNCS[partner])

    meta = {
        "type": "combo",
        "primary": primary,
        "partner": partner,
        "primary_area": float(m1.mean()),
        "partner_area": float(m2.mean()),
        "non_overlap": bool(not (m1 & m2).any())
    }
    return out, meta

def build_pool(source_dir: Optional[str], cache_dir: Path, resize: int) -> List[Image.Image]:
    if source_dir:
        imgs = load_source_images_from_dir(Path(source_dir))
        if resize and resize > 0:
            imgs = [img.resize((resize, resize), Image.BICUBIC) for img in imgs]
        return imgs
    return download_cifar10(cache_dir, resize=resize)

def generate_dataset(
    out_dir: Path,
    per_class: int = 2000,
    single_per_class: int = 1800,
    combo_per_class: int = 200,
    resize: int = 128,
    seed: int = 42,
    source_dir: Optional[str] = None
):
    random.seed(seed)
    np.random.seed(seed)

    classes = list(NOISE_FUNCS.keys())
    for c in classes:
        ensure_dir(out_dir / c)

    pool = build_pool(source_dir, Path("./_cache"), resize)
    n_pool = len(pool)
    if n_pool == 0:
        raise RuntimeError("Kaynak havuz boş.")

    # her sınıf için üretim
    for primary in classes:
        print(f"[{primary}] üretim → single:{single_per_class}, combo:{combo_per_class}")
        # tek gürültü
        for idx in range(single_per_class):
            src = pool[(idx + seed) % n_pool]
            out_img, meta = generate_single(src, primary)
            fname = f"{primary}_single_{idx:05d}_area{meta['mask_area']:.2f}.png"
            out_img.save(out_dir / primary / fname)

        # kombinasyon: primary + partner (dominant primary)
        partners = [c for c in classes if c != primary]
        for idx in range(combo_per_class):
            src = pool[(idx * 7 + seed) % n_pool]
            partner = random.choice(partners)
            out_img, meta = generate_combo(src, primary, partner)
            fname = (
                f"{primary}_combo_{partner}_{idx:05d}"
                f"_p{meta['primary_area']:.2f}_s{meta['partner_area']:.2f}.png"
            )
            out_img.save(out_dir / primary / fname)

        print(f"[{primary}] tamamlandı: {per_class} görsel kaydedildi.")

    total = len(classes) * per_class
    print(f"[OK] Toplam {total} görsel -> {out_dir}")

# ----------------------------
# CLI
# ----------------------------
def parse_args():
    ap = argparse.ArgumentParser(description="Noisy dataset (single + dominant-combo, mask-based, no overlap)")
    ap.add_argument("--out_dir", type=str, default="./noisy_dataset", help="Çıktı klasörü")
    ap.add_argument("--per_class", type=int, default=2000, help="Sınıf başına toplam adet")
    ap.add_argument("--single_per_class", type=int, default=1800, help="Sınıf başına tek gürültü adedi")
    ap.add_argument("--combo_per_class", type=int, default=200, help="Sınıf başına kombinasyon adedi")
    ap.add_argument("--resize", type=int, default=128, help="Hedef çözünürlük (px). 0=kapalı")
    ap.add_argument("--seed", type=int, default=42)
    ap.add_argument("--source_dir", type=str, default=None, help="Kendi temiz görsellerin (varsa)")
    return ap.parse_args()

def main():
    args = parse_args()
    if args.single_per_class + args.combo_per_class != args.per_class:
        raise ValueError("single_per_class + combo_per_class = per_class olmalı.")
    out_dir = Path(args.out_dir); ensure_dir(out_dir)
    generate_dataset(
        out_dir=out_dir,
        per_class=args.per_class,
        single_per_class=args.single_per_class,
        combo_per_class=args.combo_per_class,
        resize=args.resize,
        seed=args.seed,
        source_dir=args.source_dir
    )

if __name__ == "__main__":
    import sys
    if "google.colab" in sys.modules:
        # Colab için manuel argüman ver
        args = argparse.Namespace(
            out_dir="./noisy_dataset",
            per_class=2000,
            single_per_class=1800,
            combo_per_class=200,
            resize=128,
            seed=42,
            source_dir=None
        )
        out_dir = Path(args.out_dir)
        ensure_dir(out_dir)
        generate_dataset(
            out_dir=out_dir,
            per_class=args.per_class,
            single_per_class=args.single_per_class,
            combo_per_class=args.combo_per_class,
            resize=args.resize,
            seed=args.seed,
            source_dir=args.source_dir
        )
    else:
        main()



100%|██████████| 170M/170M [00:04<00:00, 37.6MB/s]
/tmp/ipython-input-1016166834.py:62: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  imgs.append(Image.fromarray(arr, mode="RGB"))


[gaussian] üretim → single:1800, combo:200


/tmp/ipython-input-1016166834.py:72: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(arr, mode="RGB")


[gaussian] tamamlandı: 2000 görsel kaydedildi.
[poisson] üretim → single:1800, combo:200
[poisson] tamamlandı: 2000 görsel kaydedildi.
[salt] üretim → single:1800, combo:200
[salt] tamamlandı: 2000 görsel kaydedildi.
[speckle] üretim → single:1800, combo:200
[speckle] tamamlandı: 2000 görsel kaydedildi.
[perlin] üretim → single:1800, combo:200
[perlin] tamamlandı: 2000 görsel kaydedildi.
[OK] Toplam 10000 görsel -> noisy_dataset


In [ ]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
from pathlib import Path
import numpy as np


def set_seed(seed=42):
    """Tüm random seed'leri sabitler"""
    random.seed(seed)
    np.random.seed(seed)


def create_split_directories(output_path):
    """Train/Test/Val klasörlerini oluşturur"""
    splits = ['train', 'test', 'val']
    classes = ['gaussian', 'perlin', 'poisson', 'salt', 'speckle']

    for split in splits:
        for class_name in classes:
            dir_path = Path(output_path) / split / class_name
            dir_path.mkdir(parents=True, exist_ok=True)

    print(f"Klasör yapısı oluşturuldu: {output_path}")


def get_files_per_class(dataset_path):
    """Her sınıf için dosya listelerini alır"""
    classes = ['gaussian', 'perlin', 'poisson', 'salt', 'speckle']
    class_files = {}

    for class_name in classes:
        class_path = Path(dataset_path) / class_name
        if class_path.exists():
            files = [f for f in class_path.glob('*') if
                     f.is_file() and f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']]
            class_files[class_name] = files
            print(f"{class_name}: {len(files)} dosya")
        else:
            print(f"Uyarı: {class_path} klasörü bulunamadı!")
            class_files[class_name] = []

    return class_files


def split_and_copy_files(class_files, output_path, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    """
    Dosyaları train/val/test olarak böler ve kopyalar
    train_ratio + val_ratio + test_ratio = 1.0 olmalı
    """
    set_seed(seed)

    split_info = {
        'train': {},
        'val': {},
        'test': {}
    }

    total_files = 0

    for class_name, files in class_files.items():
        if len(files) == 0:
            print(f"Uyarı: {class_name} sınıfında dosya yok!")
            continue

        total_files += len(files)

        # Önce train ve geçici (val+test) ayırımı
        train_files, temp_files = train_test_split(
            files,
            train_size=train_ratio,
            random_state=seed,
            shuffle=True
        )

        # Geçici dosyaları val ve test olarak ayır
        # val_ratio ve test_ratio'yu geçici dosyalar üzerinden hesapla
        val_size = val_ratio / (val_ratio + test_ratio)

        val_files, test_files = train_test_split(
            temp_files,
            train_size=val_size,
            random_state=seed,
            shuffle=True
        )

        # Dosya sayılarını kaydet
        split_info['train'][class_name] = len(train_files)
        split_info['val'][class_name] = len(val_files)
        split_info['test'][class_name] = len(test_files)

        # Dosyaları kopyala
        splits_data = {
            'train': train_files,
            'val': val_files,
            'test': test_files
        }

        for split_name, file_list in splits_data.items():
            dest_dir = Path(output_path) / split_name / class_name

            for i, file_path in enumerate(file_list):
                # Hedef dosya adını oluştur (çakışmaları önlemek için)
                dest_file = dest_dir / f"{class_name}_{i:04d}{file_path.suffix}"
                shutil.copy2(file_path, dest_file)

        print(f"{class_name}: Train={len(train_files)}, Val={len(val_files)}, Test={len(test_files)}")

    # Özet bilgileri yazdır
    print("\n" + "=" * 50)
    print("DATASET BÖLÜNME ÖZETİ")
    print("=" * 50)

    for split_name in ['train', 'val', 'test']:
        total_split = sum(split_info[split_name].values())
        print(f"\n{split_name.upper()}:")
        for class_name in split_info[split_name]:
            count = split_info[split_name][class_name]
            percentage = (count / total_files) * 100 if total_files > 0 else 0
            print(f"  {class_name}: {count} dosya ({percentage:.1f}%)")
        print(f"  TOPLAM: {total_split} dosya")

    print(f"\nTOPLAM DOSYA SAYISI: {total_files}")
    print(f"Seed kullanıldı: {seed}")

    return split_info


def save_split_info(split_info, output_path, seed):
    """Bölünme bilgilerini txt dosyasına kaydeder"""
    info_file = Path(output_path) / "dataset_split_info.txt"

    with open(info_file, 'w', encoding='utf-8') as f:
        f.write("DATASET BÖLÜNME BİLGİLERİ\n")
        f.write("=" * 50 + "\n")
        f.write(f"Kullanılan Seed: {seed}\n")
        f.write(f"Oluşturulma Tarihi: {Path(__file__).stat().st_mtime}\n\n")

        total_files = sum(sum(class_dict.values()) for class_dict in split_info.values())

        for split_name in ['train', 'val', 'test']:
            total_split = sum(split_info[split_name].values())
            f.write(f"{split_name.upper()}:\n")
            for class_name in ['gaussian', 'perlin', 'poisson', 'salt', 'speckle']:
                count = split_info[split_name].get(class_name, 0)
                percentage = (count / total_files) * 100 if total_files > 0 else 0
                f.write(f"  {class_name}: {count} dosya ({percentage:.1f}%)\n")
            f.write(f"  TOPLAM: {total_split} dosya\n\n")

        f.write(f"GENEL TOPLAM: {total_files} dosya\n")

    print(f"Bölünme bilgileri kaydedildi: {info_file}")


def main():
    # Konfigürasyon
    SEED = 42
    INPUT_DATASET_PATH = "noisy_dataset"  # Ana dataset klasörünüz
    OUTPUT_DATASET_PATH = "split_dataset"  # Bölünmüş dataset kayıt yeri

    # Bölünme oranları (toplamı 1.0 olmalı)
    TRAIN_RATIO = 0.7  # %70 eğitim
    VAL_RATIO = 0.15  # %15 doğrulama
    TEST_RATIO = 0.15  # %15 test

    print("Gürültü Sınıflandırması Dataset Bölme Script'i")
    print("=" * 50)
    print(f"Kaynak Dataset: {INPUT_DATASET_PATH}")
    print(f"Hedef Dataset: {OUTPUT_DATASET_PATH}")
    print(f"Train: {TRAIN_RATIO * 100}%, Val: {VAL_RATIO * 100}%, Test: {TEST_RATIO * 100}%")
    print(f"Seed: {SEED}")
    print()

    # Seed'i sabitler
    set_seed(SEED)

    # Kaynak dataset kontrolü
    if not Path(INPUT_DATASET_PATH).exists():
        print(f"Hata: {INPUT_DATASET_PATH} klasörü bulunamadı!")
        return

    # Hedef klasör yapısını oluştur
    create_split_directories(OUTPUT_DATASET_PATH)

    # Her sınıf için dosyaları al
    class_files = get_files_per_class(INPUT_DATASET_PATH)

    # Dosyaları böl ve kopyala
    split_info = split_and_copy_files(
        class_files,
        OUTPUT_DATASET_PATH,
        train_ratio=TRAIN_RATIO,
        val_ratio=VAL_RATIO,
        test_ratio=TEST_RATIO,
        seed=SEED
    )

    # Bölünme bilgilerini kaydet
    save_split_info(split_info, OUTPUT_DATASET_PATH, SEED)

    print("\nDataset bölme işlemi tamamlandı!")


if __name__ == "__main__":
    main()

Gürültü Sınıflandırması Dataset Bölme Script'i
Kaynak Dataset: noisy_dataset
Hedef Dataset: split_dataset
Train: 70.0%, Val: 15.0%, Test: 15.0%
Seed: 42

Klasör yapısı oluşturuldu: split_dataset
gaussian: 2000 dosya
perlin: 2000 dosya
poisson: 2000 dosya
salt: 2000 dosya
speckle: 2000 dosya
gaussian: Train=1400, Val=300, Test=300
perlin: Train=1400, Val=300, Test=300
poisson: Train=1400, Val=300, Test=300
salt: Train=1400, Val=300, Test=300
speckle: Train=1400, Val=300, Test=300

DATASET BÖLÜNME ÖZETİ

TRAIN:
  gaussian: 1400 dosya (14.0%)
  perlin: 1400 dosya (14.0%)
  poisson: 1400 dosya (14.0%)
  salt: 1400 dosya (14.0%)
  speckle: 1400 dosya (14.0%)
  TOPLAM: 7000 dosya

VAL:
  gaussian: 300 dosya (3.0%)
  perlin: 300 dosya (3.0%)
  poisson: 300 dosya (3.0%)
  salt: 300 dosya (3.0%)
  speckle: 300 dosya (3.0%)
  TOPLAM: 1500 dosya

TEST:
  gaussian: 300 dosya (3.0%)
  perlin: 300 dosya (3.0%)
  poisson: 300 dosya (3.0%)
  salt: 300 dosya (3.0%)
  speckle: 300 dosya (3.0%)
  TOPLAM:

NameError: name '__file__' is not defined

In [ ]:
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F


class DeepCNN(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.gap = nn.AdaptiveAvgPool2d((1, 1))  # 128 x 1 x 1
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(128, 512), nn.BatchNorm1d(512), nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 64), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Dropout(0.05),
            nn.Linear(64, 32), nn.BatchNorm1d(32), nn.ReLU(),
            nn.Linear(32, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.gap(x).flatten(1)
        return self.classifier(x)


# ResNet34 (pretrained & fine-tuned)
class ResNet34Pretrained(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        m = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        m.fc = nn.Linear(m.fc.in_features, num_classes)
        self.model = m

    def forward(self, x):
        return self.model(x)


# ResNet34 (scratch)
class ResNet34Scratch(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        m = models.resnet34(weights=None)
        m.fc = nn.Linear(m.fc.in_features, num_classes)
        self.model = m

    def forward(self, x):
        return self.model(x)


# ResNet50 (pretrained)
class ResNet50Pretrained(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        m = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        m.fc = nn.Linear(m.fc.in_features, num_classes)
        self.model = m

    def forward(self, x):
        return self.model(x)


# ResNet50 (scratch)
class ResNet50Scratch(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        m = models.resnet50(weights=None)
        m.fc = nn.Linear(m.fc.in_features, num_classes)
        self.model = m

    def forward(self, x):
        return self.model(x)


# EfficientNetB0
class EfficientNetB0Custom(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        m = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        in_features = m.classifier[1].in_features
        m.classifier[1] = nn.Linear(in_features, num_classes)
        self.model = m

    def forward(self, x):
        return self.model(x)


# MobileNetV2
class MobileNetV2Custom(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        m = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
        m.classifier[1] = nn.Linear(m.classifier[1].in_features, num_classes)
        self.model = m

    def forward(self, x):
        return self.model(x)


# Deeper CNN with more layers
class DeeperCNN(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2),

            # Block 2
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),

            # Block 3
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2),

            # Block 4
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d(2),

            # Block 5
            nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.gap = nn.AdaptiveAvgPool2d((1, 1))  # 512 x 1 x 1

        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(512, 1024), nn.BatchNorm1d(1024), nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.ReLU(),
            nn.Dropout(0.15),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Dropout(0.05),
            nn.Linear(128, 64), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Linear(64, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.gap(x).flatten(1)
        return self.classifier(x)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import time
import os
from pathlib import Path
import random
from datetime import datetime
from models import DeepCNN, ResNet34Pretrained, ResNet34Scratch, EfficientNetB0Custom, MobileNetV2Custom, DeeperCNN, ResNet50Pretrained


def set_seed(seed=42):
    """Tüm random seed'leri sabitler"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def get_device():
    """GPU/CPU device seçimi"""
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f" GPU kullanılıyor: {torch.cuda.get_device_name(0)}")
        print(f"CUDA Version: {torch.version.cuda}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024 ** 3:.1f} GB")
        print(f"PyTorch CUDA Support: {torch.backends.cudnn.enabled}")

        # GPU'ya test tensörü gönder
        test_tensor = torch.randn(1, 1).to(device)
        print(f"Test tensor device: {test_tensor.device}")

    else:
        device = torch.device('cpu')
        print(" CUDA kullanılamıyor, CPU kullanılıyor")
        print("Nedenleri:")
        print("- CUDA yüklü değil")
        print("- PyTorch CUDA versiyonu yüklü değil")
        print("- GPU driver sorunu")
    return device


def get_data_transforms():
    """Veri dönüşüm pipeline'ları"""
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    return train_transform, val_test_transform


def create_data_loaders(dataset_path, batch_size=32, num_workers=4):
    """Data loader'ları oluşturur"""
    train_transform, val_test_transform = get_data_transforms()

    # Datasetleri yükle
    train_dataset = datasets.ImageFolder(
        root=Path(dataset_path) / 'train',
        transform=train_transform
    )

    val_dataset = datasets.ImageFolder(
        root=Path(dataset_path) / 'val',
        transform=val_test_transform
    )

    test_dataset = datasets.ImageFolder(
        root=Path(dataset_path) / 'test',
        transform=val_test_transform
    )

    # Data loader'ları oluştur
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    # Sınıf isimleri
    class_names = train_dataset.classes
    print(f"Sınıflar: {class_names}")
    print(f"Train: {len(train_dataset)} örnek")
    print(f"Val: {len(val_dataset)} örnek")
    print(f"Test: {len(test_dataset)} örnek")

    return train_loader, val_loader, test_loader, class_names


def train_epoch(model, train_loader, criterion, optimizer, device):
    """Bir epoch eğitim"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        if (batch_idx + 1) % 50 == 0:
            print(f'    Batch {batch_idx + 1}/{len(train_loader)}, '
                  f'Loss: {running_loss / (batch_idx + 1):.4f}, '
                  f'Acc: {100. * correct / total:.2f}%')

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def validate(model, val_loader, criterion, device):
    """Validasyon"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
            output = model(data)
            loss = criterion(output, target)

            running_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    val_loss = running_loss / len(val_loader)
    val_acc = 100. * correct / total
    return val_loss, val_acc


def test_model(model, test_loader, device, class_names):
    """Test ve classification report"""
    model.eval()
    all_preds = []
    all_targets = []
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
            output = model(data)
            _, predicted = output.max(1)

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_acc = 100. * correct / total

    # Classification report
    report = classification_report(
        all_targets,
        all_preds,
        target_names=class_names,
        digits=4
    )

    return test_acc, report, all_preds, all_targets


def train_model(model_class, model_name, train_loader, val_loader, test_loader,
                class_names, device, epochs=25, lr=0.001):
    print(f"\n{'=' * 60}")
    print(f"EĞİTİM BAŞLADI: {model_name}")
    print(f"{'=' * 60}")

    model = model_class(num_classes=len(class_names)).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

    train_losses, train_accs, val_losses, val_accs = [], [], [], []

    # İlk değeri çok düşük başlat ve en az 1 kez set edileceğinden emin ol
    best_val_acc = -1.0
    best_model_state = None

    start_time = time.time()

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")

        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss); train_accs.append(train_acc)

        val_loss, val_acc = validate(model, val_loader, criterion, device)
        val_losses.append(val_loss); val_accs.append(val_acc)

        scheduler.step()

        print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
        print(f"  Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()  # kopya almak şart değil, save sırasında serialize edilir
            print(f"  *** Yeni en iyi model! Val Acc: {val_acc:.2f}% ***")

    # Eğer herhangi bir iyileşme olmadıysa (edge case), mevcut ağırlıkları kullan
    if best_model_state is None:
        best_model_state = model.state_dict()

    model.load_state_dict(best_model_state)

    training_time = time.time() - start_time
    print(f"\nEğitim tamamlandı! Süre: {training_time:.2f} saniye")
    print(f"En iyi validasyon accuracy: {best_val_acc:.2f}%")

    print("\nTest değerlendirmesi yapılıyor...")
    test_acc, report, test_preds, test_targets = test_model(model, test_loader, device, class_names)
    print(f"Test Accuracy: {test_acc:.2f}%")

    results = {
        'model_name': model_name,
        'train_losses': train_losses,
        'train_accs': train_accs,
        'val_losses': val_losses,
        'val_accs': val_accs,
        'best_val_acc': best_val_acc,
        'test_acc': test_acc,
        'test_report': report,
        'training_time': training_time,
        'test_preds': test_preds,
        'test_targets': test_targets,
        'best_model_state': best_model_state,   # <-- EKLENDİ
    }

    return model, results



def save_results(results, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    model_name = results['model_name']
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # classification report
    report_path = Path(output_dir) / f"{model_name}_classification_report_{timestamp}.txt"
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(f"MODEL: {model_name}\n")
        f.write(f"TARIH: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("=" * 60 + "\n")
        f.write(f"En İyi Validasyon Accuracy: {results['best_val_acc']:.4f}%\n")
        f.write(f"Test Accuracy: {results['test_acc']:.4f}%\n")
        f.write(f"Eğitim Süresi: {results['training_time']:.2f} saniye\n")
        f.write("\nCLASSIFICATION REPORT:\n")
        f.write("=" * 60 + "\n")
        f.write(results['test_report'])

    print(f"Classification report kaydedildi: {report_path}")

    # model ağırlıkları
    model_state = results.get('best_model_state', None)
    model_path = None
    if model_state is not None:
        model_path = Path(output_dir) / f"{model_name}_best_model_{timestamp}.pth"
        torch.save(model_state, model_path)
        print(f"Model kaydedildi: {model_path}")
    else:
        print("Uyarı: best_model_state bulunamadı, model kaydedilmedi.")

    return report_path, model_path



def main():
    # Konfigürasyon
    SEED = 42
    DATASET_PATH = "split_dataset"
    OUTPUT_DIR = "training_results"
    BATCH_SIZE = 32
    EPOCHS = 15
    LEARNING_RATE = 0.001
    NUM_WORKERS = 4

    # Seed sabitler
    set_seed(SEED)

    # Device seçimi
    device = get_device()

    # Veri yükleyicilerini oluştur
    print("Veri yükleyicileri hazırlanıyor...")
    train_loader, val_loader, test_loader, class_names = create_data_loaders(
        DATASET_PATH, BATCH_SIZE, NUM_WORKERS
    )

    # Eğitilecek modeller
    models_to_train = [
 	    (DeepCNN, "DeepCNN"),
      (ResNet34Pretrained, "ResNet34_Pretrained"),
      (ResNet34Scratch, "ResNet34_Scratch"),
      (EfficientNetB0Custom, "EfficientNetB0"),
      (MobileNetV2Custom, "MobileNetV2"),
      (DeepCNN, "DeeperCNN"),
      (ResNet50Pretrained, "ResNet50_Pretrained"),
      (ResNet50Scratch, "ResNet50_Scratch"),
    ]

    # Tüm sonuçları sakla
    all_results = []

    print(f"\nToplam {len(models_to_train)} model eğitilecek")
    print(f"Epochs: {EPOCHS}, Batch Size: {BATCH_SIZE}, LR: {LEARNING_RATE}")

    for i, (model_class, model_name) in enumerate(models_to_train):
        print(f"\n Model {i + 1}/{len(models_to_train)}: {model_name}")

        try:
            model, results = train_model(
                model_class, model_name,
                train_loader, val_loader, test_loader,
                class_names, device, EPOCHS, LEARNING_RATE
            )

            # Sonuçları kaydet (rapor + model dosyası)
            report_path, model_path = save_results(results, OUTPUT_DIR)
            results['report_path'] = report_path
            results['model_path'] = model_path
            all_results.append(results)

            # Memory temizle
            del model
            torch.cuda.empty_cache()

        except Exception as e:
            print(f" {model_name} eğitiminde hata: {str(e)}")
            continue

    # Özet rapor
    print(f"\n{'=' * 80}")
    print("TÜM MODELLERİN EĞİTİM ÖZETİ")
    print(f"{'=' * 80}")

    for result in all_results:
        print(f"{result['model_name']:<20} - "
              f"Val: {result['best_val_acc']:.2f}% | "
              f"Test: {result['test_acc']:.2f}% | "
              f"Süre: {result['training_time']:.1f}s")

    # En iyi modeli bul
    if all_results:
        best_result = max(all_results, key=lambda x: x['test_acc'])
        print(f"\n EN İYİ MODEL: {best_result['model_name']} "
              f"(Test Acc: {best_result['test_acc']:.2f}%)")
        print(f" Kaydedilen dosya: {best_result['model_path']}")

    print(f"\nTüm sonuçlar kaydedildi: {OUTPUT_DIR}")



if __name__ == "__main__":

    main()


 GPU kullanılıyor: Tesla T4
CUDA Version: 12.6
GPU Memory: 14.7 GB
PyTorch CUDA Support: True
Test tensor device: cuda:0
Veri yükleyicileri hazırlanıyor...
Sınıflar: ['gaussian', 'perlin', 'poisson', 'salt', 'speckle']
Train: 7000 örnek
Val: 1500 örnek
Test: 1500 örnek

Toplam 2 model eğitilecek
Epochs: 15, Batch Size: 32, LR: 0.001

 Model 1/2: DeeperCNN

EĞİTİM BAŞLADI: DeeperCNN

Epoch 1/15
Learning Rate: 0.001000
    Batch 50/219, Loss: 1.4924, Acc: 32.31%
    Batch 100/219, Loss: 1.3881, Acc: 39.25%
    Batch 150/219, Loss: 1.2981, Acc: 44.17%
    Batch 200/219, Loss: 1.2141, Acc: 48.61%


KeyboardInterrupt: 

In [ ]:
# test_models.py
# -*- coding: utf-8 -*-

import os
from pathlib import Path
from datetime import datetime
import argparse
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# (Opsiyonel) seaborn varsa kullan; yoksa matplotlib ile devam
try:
    import seaborn as sns
    _HAS_SNS = True
except Exception:
    _HAS_SNS = False

# ---- Modeller ----
from models import (
    DeepCNN,
    ResNet34Pretrained,
    ResNet34Scratch,
    EfficientNetB0Custom,
    MobileNetV2Custom,
    DeeperCNN,
    ResNet50Pretrained
)

# -----------------------------
# Yardımcılar
# -----------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"GPU kullanılıyor: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device('cpu')
        print("CPU kullanılıyor")
    return device


def get_test_transform(img_size=224):
    return transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])


def create_test_loader(dataset_path, batch_size=32, num_workers=4, img_size=224):
    dataset_path = Path(dataset_path)
    test_dir = dataset_path / 'test'
    if not test_dir.exists():
        raise FileNotFoundError(
            f"'test' klasörü bulunamadı: {test_dir}\n"
            f"Yapı şu formatta olmalı:\n"
            f"{dataset_path}/test/<class_name>/*.png"
        )

    test_dataset = datasets.ImageFolder(
        root=test_dir,
        transform=get_test_transform(img_size)
    )
    # Windows/Colab ortamında worker sayısını güvenli tut
    if num_workers is None or num_workers < 0:
        num_workers = 0

    pin_memory = torch.cuda.is_available()
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory
    )

    class_names = test_dataset.classes
    print(f"Test sınıfları: {class_names}")
    print(f"Test örnekleri: {len(test_dataset)}")
    return test_loader, class_names


def _extract_state_dict(ckpt):
    """
    Farklı kaydetme formatlarını destekler:
    - Doğrudan state_dict
    - {'state_dict': ...} (Lightning tipik)
    - {'model_state_dict': ...} (PyTorch common)
    """
    if isinstance(ckpt, dict):
        for key in ['state_dict', 'model_state_dict']:
            if key in ckpt and isinstance(ckpt[key], dict):
                return ckpt[key]
    return ckpt  # zaten state_dict olabilir


def load_model(model_class, model_path, num_classes, device):
    model = model_class(num_classes=num_classes).to(device)

    if model_path is None:
        print("Checkpoint verilmedi, model rastgele ağırlıklarla yüklendi (sadece değerlendirme).")
        model.eval()
        return model

    try:
        ckpt = torch.load(model_path, map_location=device)
        state_dict = _extract_state_dict(ckpt)

        # Lightning tarzı 'model.' veya 'module.' ön eklerini temizle
        cleaned = {}
        for k, v in state_dict.items():
            if k.startswith('model.'):
                cleaned[k[len('model.'):]] = v
            elif k.startswith('module.'):
                cleaned[k[len('module.'):]] = v
            else:
                cleaned[k] = v

        missing, unexpected = model.load_state_dict(cleaned, strict=False)
        if missing:
            print(f"Uyarı: Eksik anahtarlar: {missing}")
        if unexpected:
            print(f"Uyarı: Beklenmeyen anahtarlar: {unexpected}")

        model.eval()
        print(f"Model başarıyla yüklendi: {model_path}")
        return model
    except Exception as e:
        print(f"Model yüklenirken hata: {str(e)}")
        return None


@torch.no_grad()
def test_model_detailed(model, test_loader, device, class_names, model_name):
    model.eval()
    all_preds, all_targets, all_probs = [], [], []
    correct, total = 0, 0
    class_correct = {i: 0 for i in range(len(class_names))}
    class_total = {i: 0 for i in range(len(class_names))}

    print(f"\n{model_name} test ediliyor...")

    for batch_idx, (data, target) in enumerate(test_loader):
        data = data.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)

        output = model(data)
        probs = torch.softmax(output, dim=1)
        predicted = torch.argmax(output, dim=1)

        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(target.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

        total += target.size(0)
        correct += (predicted == target).sum().item()

        for i in range(target.size(0)):
            label = target[i].item()
            class_total[label] += 1
            if predicted[i] == label:
                class_correct[label] += 1

        if (batch_idx + 1) % 20 == 0:
            print(f"  Batch {batch_idx + 1}/{len(test_loader)} işlendi")

    overall_acc = 100.0 * correct / max(total, 1)

    class_accuracies = {}
    for i, cname in enumerate(class_names):
        denom = max(class_total[i], 1)
        class_accuracies[cname] = 100.0 * class_correct[i] / denom

    report = classification_report(
        all_targets, all_preds, target_names=class_names, digits=4, zero_division=0
    )
    cm = confusion_matrix(all_targets, all_preds, labels=list(range(len(class_names))))

    return {
        'model_name': model_name,
        'overall_accuracy': overall_acc,
        'class_accuracies': class_accuracies,
        'classification_report': report,
        'confusion_matrix': cm,
        'predictions': all_preds,
        'targets': all_targets,
        'probabilities': all_probs,
        'class_names': class_names
    }


def plot_confusion_matrix(cm, class_names, model_name, save_path=None):
    plt.figure(figsize=(10, 8))
    if _HAS_SNS:
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names, yticklabels=class_names)
    else:
        plt.imshow(cm)
        plt.xticks(ticks=range(len(class_names)), labels=class_names, rotation=45, ha='right')
        plt.yticks(ticks=range(len(class_names)), labels=class_names)
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                plt.text(j, i, str(cm[i, j]),
                         ha='center', va='center')
        plt.colorbar()
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    if save_path:
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Confusion matrix kaydedildi: {save_path}")
    plt.show()
    plt.close()


def save_detailed_results(results, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    model_name = results['model_name']
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_path = Path(output_dir) / f"{model_name}_detailed_test_report_{timestamp}.txt"

    with open(report_path, 'w', encoding='utf-8') as f:
        f.write("DETAYLI TEST RAPORU\n")
        f.write("=" * 60 + "\n")
        f.write(f"Model: {model_name}\n")
        f.write(f"Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Test Accuracy: {results['overall_accuracy']:.4f}%\n\n")

        f.write("SINIF BAZINDA DOĞRULUK ORANLARI:\n")
        f.write("-" * 40 + "\n")
        for cname, acc in results['class_accuracies'].items():
            f.write(f"{cname:<15}: {acc:.2f}%\n")
        f.write("\n")

        f.write("CLASSIFICATION REPORT:\n")
        f.write("-" * 40 + "\n")
        f.write(results['classification_report'])
        f.write("\n")

        f.write("CONFUSION MATRIX:\n")
        f.write("-" * 40 + "\n")
        cm = results['confusion_matrix']

        f.write(f"{'':>12}")
        for cname in results['class_names']:
            f.write(f"{cname[:8]:>8}")
        f.write("\n")

        for i, cname in enumerate(results['class_names']):
            f.write(f"{cname[:10]:>10}  ")
            for j in range(len(results['class_names'])):
                f.write(f"{cm[i, j]:>8}")
            f.write("\n")
        f.write("\n")

        f.write("İSTATİSTİKLER:\n")
        f.write("-" * 40 + "\n")
        preds = np.array(results['predictions'])
        targs = np.array(results['targets'])
        f.write(f"Toplam test örneği: {len(targs)}\n")
        f.write(f"Doğru tahmin: {(preds == targs).sum()}\n")
        f.write(f"Yanlış tahmin: {(preds != targs).sum()}\n")

    print(f"Detaylı test raporu kaydedildi: {report_path}")

    cm_path = Path(output_dir) / f"{model_name}_confusion_matrix_{timestamp}.png"
    plot_confusion_matrix(results['confusion_matrix'], results['class_names'], model_name, cm_path)
    return report_path, cm_path


def compare_models(all_results, output_dir):
    if not all_results:
        return None

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    comparison_path = Path(output_dir) / f"model_comparison_{timestamp}.txt"

    with open(comparison_path, 'w', encoding='utf-8') as f:
        f.write("MODEL KARŞILAŞTIRMA RAPORU\n")
        f.write("=" * 60 + "\n")
        f.write(f"Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Test edilen model sayısı: {len(all_results)}\n\n")

        sorted_results = sorted(all_results, key=lambda x: x['overall_accuracy'], reverse=True)

        f.write("GENEL PERFORMANS SIRALAMASI:\n")
        f.write("-" * 40 + "\n")
        for i, r in enumerate(sorted_results, 1):
            f.write(f"{i}. {r['model_name']:<20}: {r['overall_accuracy']:.2f}%\n")
        f.write("\n")

        class_names = all_results[0]['class_names']
        f.write("SINIF BAZINDA EN İYİ PERFORMANS:\n")
        f.write("-" * 40 + "\n")
        for cname in class_names:
            best_model, best_acc = "", 0.0
            for r in all_results:
                acc = r['class_accuracies'][cname]
                if acc > best_acc:
                    best_acc, best_model = acc, r['model_name']
            f.write(f"{cname:<15}: {best_model:<20} ({best_acc:.2f}%)\n")
        f.write("\n")

        f.write("DETAYLI KARŞILAŞTIRMA:\n")
        f.write("-" * 80 + "\n")
        f.write(f"{'Model':<20}{'Overall':<10}")
        for cname in class_names:
            f.write(f"{cname[:8]:<10}")
        f.write("\n")
        f.write("-" * 80 + "\n")

        for r in sorted_results:
            f.write(f"{r['model_name']:<20}{r['overall_accuracy']:.2f}%{'':>4}")
            for cname in class_names:
                f.write(f"{r['class_accuracies'][cname]:.2f}%{'':>4}")
            f.write("\n")

    print(f"Model karşılaştırma raporu kaydedildi: {comparison_path}")
    return comparison_path


# -----------------------------
# Argümanlar
# -----------------------------
def build_parser():
    ap = argparse.ArgumentParser(description="Modelleri test et ve karşılaştır")
    ap.add_argument("--dataset_path", type=str, default="split_dataset",
                    help="train/val/test alt klasörleri olan kök dizin. Burada sadece 'test' kullanılır.")
    ap.add_argument("--output_dir", type=str, default="test_results",
                    help="Rapor ve görseller için çıktı klasörü")
    ap.add_argument("--batch_size", type=int, default=32)
    ap.add_argument("--workers", type=int, default=4)
    ap.add_argument("--img_size", type=int, default=224)
    ap.add_argument("--seed", type=int, default=42)
    # Hangi modeller test edilsin
    ap.add_argument("--models", type=str, default="DeepCNN,ResNet34Pretrained,ResNet34Scratch,EfficientNetB0Custom,MobileNetV2Custom",
                    help="Virgülle ayrık model listesi")
    # Checkpoint klasörü (içinde model_adi.pth varsa otomatik alınır)
    ap.add_argument("--ckpt_dir", type=str, default=None,
                    help="Checkpoint klasörü. Örn: ckpts/ (içinde DeepCNN.pth, ResNet34Pretrained.pth vb.)")
    return ap


def resolve_model_list(models_arg: str):
    name_to_class = {
        "DeepCNN": DeepCNN,
        "ResNet34Pretrained": ResNet34Pretrained,
        "ResNet34Scratch": ResNet34Scratch,
        "EfficientNetB0Custom": EfficientNetB0Custom,
        "MobileNetV2Custom": MobileNetV2Custom,
    }
    names = [x.strip() for x in models_arg.split(",") if x.strip()]
    resolved = []
    for n in names:
        if n not in name_to_class:
            print(f"Uyarı: Bilinmeyen model adı atlandı -> {n}")
            continue
        resolved.append((n, name_to_class[n]))
    return resolved


def find_checkpoint(ckpt_dir: Path, model_name: str):
    """
    ckpt_dir / f"{model_name}.pth" varsa döndür, yoksa None
    """
    if ckpt_dir is None:
        return None
    cand = ckpt_dir / f"{model_name}.pth"
    return str(cand) if cand.exists() else None


def main():
    parser = build_parser()
    # Notebook/Colab'ta gelen tanınmayan argümanları yok say
    args, _ = parser.parse_known_args()

    set_seed(args.seed)
    device = get_device()

    test_loader, class_names = create_test_loader(
        args.dataset_path, args.batch_size, args.workers, args.img_size
    )

    model_specs = resolve_model_list(args.models)
    if not model_specs:
        raise ValueError("Test edilecek en az bir model ismi verilmeli.")

    ckpt_dir = Path(args.ckpt_dir) if args.ckpt_dir else None

    all_results = []
    print(f"Test başlıyor - {len(model_specs)} model")
    print(f"Test örnekleri: {len(test_loader.dataset)}")
    print(f"Sınıflar: {class_names}")

    for model_name, model_cls in model_specs:
        try:
            ckpt_path = find_checkpoint(ckpt_dir, model_name) if ckpt_dir else None
            if ckpt_path:
                model = load_model(model_cls, ckpt_path, len(class_names), device)
                if model is None:
                    continue
            else:
                print(f"\n⚠ {model_name}: checkpoint bulunamadı → rastgele ağırlıklarla test edilecek.")
                model = model_cls(num_classes=len(class_names)).to(device)
                model.eval()

            results = test_model_detailed(model, test_loader, device, class_names, model_name)
            all_results.append(results)
            save_detailed_results(results, args.output_dir)

            del model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            print(f"{model_name} - Test Accuracy: {results['overall_accuracy']:.2f}%")

        except Exception as e:
            print(f"{model_name} test edilirken hata: {str(e)}")
            continue

    if len(all_results) > 1:
        print("\nModel karşılaştırması yapılıyor...")
        compare_models(all_results, args.output_dir)

    print("\n" + "=" * 60)
    print("TEST ÖZETİ")
    print("=" * 60)
    if all_results:
        best = max(all_results, key=lambda x: x['overall_accuracy'])
        print(f"En iyi model: {best['model_name']} ({best['overall_accuracy']:.2f}%)")
        print("\nTüm modeller:")
        for r in sorted(all_results, key=lambda x: x['overall_accuracy'], reverse=True):
            print(f"  {r['model_name']:<20}: {r['overall_accuracy']:.2f}%")
    else:
        print("Hiç model test edilemedi!")

    print(f"\nTüm sonuçlar kaydedildi: {args.output_dir}")


if __name__ == "__main__":
    main()


In [ ]:
import torch
import torch.nn as nn
import gradio as gr
import numpy as np
from PIL import Image
import json
import os
from datetime import datetime
from pathlib import Path
import pandas as pd

# Transform imports
from torchvision import transforms

# Model imports
from models import DeepCNN, ResNet34Pretrained, ResNet34Scratch, EfficientNetB0Custom, MobileNetV2Custom, ResNet50Pretrained, DeeperCNN

class ModelTester:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.class_names = ['gaussian', 'poisson', 'salt', 'speckle', 'perlin']
        self.models = {}
        self.history_file = "test_history.json"
        self.load_history()

        # ✅ Model path listesi (boş bırakabilirsin veya kendi pathlerini yazabilirsin)
        self.model_paths = {
            "DeepCNN": "C:/Users/.../DeepCNN_best_model.pth",
            "ResNet34_Pretrained": "C:/Users/.../ResNet34_Pretrained_best_model.pth",
            "ResNet34_Scratch": "C:/Users/.../ResNet34_Scratch_best_model.pth",
            "EfficientNetB0": "C:/Users/.../EfficientNetB0_best_model.pth",
            "MobileNetV2": r"C:\Users\stj.skartal\Desktop\python\noisy_classification\training_results\MobileNetV2_best_model_20250822_145431.pth"
        }

        # Image transform pipeline
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.load_models()


    def load_models(self):
        """Tüm modelleri yükle"""
        model_configs = [
            (DeepCNN, "DeepCNN"),
            (ResNet34Pretrained, "ResNet34_Pretrained"),
            (ResNet34Scratch, "ResNet34_Scratch"),
            (EfficientNetB0Custom, "EfficientNetB0"),
            (MobileNetV2Custom, "MobileNetV2")
        ]

        print("Modeller yükleniyor...")
        for model_class, model_name in model_configs:
            try:
                # ✅ self.model_paths'te kayıtlı path varsa onu kullan
                model_path = self.model_paths.get(model_name, None)

                if model_path and os.path.exists(model_path):
                    model = model_class(num_classes=len(self.class_names)).to(self.device)
                    checkpoint = torch.load(model_path, map_location=self.device)
                    model.load_state_dict(checkpoint)
                    model.eval()
                    print(f"✓ {model_name} kaydedilmiş ağırlıklarla yüklendi ({model_path})")
                else:
                    # Eğer kaydedilmiş model yoksa rastgele ağırlıklarla oluştur
                    model = model_class(num_classes=len(self.class_names)).to(self.device)
                    model.eval()
                    print(f"⚠ {model_name} rastgele ağırlıklarla oluşturuldu (eğitilmemiş)")

                self.models[model_name] = model

            except Exception as e:
                print(f"✗ {model_name} yüklenemedi: {str(e)}")


    def preprocess_image(self, image):
        """Görüntüyü model için uygun formata çevir"""
        if isinstance(image, str):
            # Eğer dosya yolu ise
            image = Image.open(image)

        # PIL Image'a çevir
        if not isinstance(image, Image.Image):
            image = Image.fromarray(image)

        # RGB'ye çevir
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Transform uygula
        tensor = self.transform(image).unsqueeze(0)  # Batch dimension ekle
        return tensor.to(self.device)

    def predict_single_model(self, model, image_tensor):
        """Tek model ile tahmin yap"""
        with torch.no_grad():
            output = model(image_tensor)
            probabilities = torch.softmax(output, dim=1)
            confidence, predicted = torch.max(probabilities, 1)

        return {
            'predicted_class': self.class_names[predicted.item()],
            'confidence': float(confidence.item() * 100),
            'probabilities': {
                self.class_names[i]: float(prob * 100)
                for i, prob in enumerate(probabilities[0].cpu().numpy())
            }
        }

    def predict_all_models(self, image):
        """Tüm modellerle tahmin yap"""
        if image is None:
            return "Lütfen bir resim yükleyin!", "", ""

        try:
            # Görüntüyü hazırla
            image_tensor = self.preprocess_image(image)

            results = {}
            predictions_text = "🤖 **MODEL TAHMİNLERİ:**\n\n"

            for model_name, model in self.models.items():
                try:
                    result = self.predict_single_model(model, image_tensor)
                    results[model_name] = result

                    predictions_text += f"**{model_name}:**\n"
                    predictions_text += f"  • Tahmin: **{result['predicted_class']}**\n"
                    predictions_text += f"  • Güven: **{result['confidence']:.1f}%**\n\n"

                except Exception as e:
                    predictions_text += f"**{model_name}:** Hata - {str(e)}\n\n"

            # Geçici sonuçları sakla
            self.temp_predictions = results

            return predictions_text, self.get_accuracy_stats(), ""

        except Exception as e:
            return f"Hata: {str(e)}", "", ""

    def load_history(self):
        """Geçmişi yükle"""
        if os.path.exists(self.history_file):
            try:
                with open(self.history_file, 'r', encoding='utf-8') as f:
                    self.history = json.load(f)
            except:
                self.history = []
        else:
            self.history = []

    def save_history(self):
        """Geçmişi kaydet"""
        # JSON serileştirme için float32'leri float'a çevir
        def convert_floats(obj):
            if isinstance(obj, dict):
                return {k: convert_floats(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [convert_floats(v) for v in obj]
            elif isinstance(obj, (np.float32, np.float64)):
                return float(obj)
            elif isinstance(obj, (np.int32, np.int64)):
                return int(obj)
            return obj

        cleaned_history = convert_floats(self.history)

        with open(self.history_file, 'w', encoding='utf-8') as f:
            json.dump(cleaned_history, f, ensure_ascii=False, indent=2)

    def submit_ground_truth(self, true_label):
        """Gerçek etiketi kaydet ve modelleri değerlendir"""
        if not hasattr(self, 'temp_predictions'):
            return "Önce bir resim yükleyip tahmin yaptırın!", ""

        if not true_label:
            return "Lütfen gerçek gürültü tipini seçin!", ""

        # Geçmişe ekle
        record = {
            'timestamp': datetime.now().isoformat(),
            'true_label': true_label,
            'predictions': self.temp_predictions
        }

        self.history.append(record)
        self.save_history()

        # Sonuçları değerlendir
        results_text = f"✅ **DOĞRULAMA SONUÇLARI** (Gerçek: **{true_label}**)\n\n"

        for model_name, prediction in self.temp_predictions.items():
            predicted = prediction['predicted_class']
            confidence = prediction['confidence']

            if predicted == true_label:
                results_text += f"✅ **{model_name}**: DOĞRU! ({confidence:.1f}%)\n"
            else:
                results_text += f"❌ **{model_name}**: YANLIŞ - {predicted} ({confidence:.1f}%)\n"

        results_text += f"\n📊 **Toplam test sayısı**: {len(self.history)}"

        # Temp predictions'ı temizle
        delattr(self, 'temp_predictions')

        return results_text, self.get_accuracy_stats()

    def get_accuracy_stats(self):
        """Doğruluk istatistiklerini hesapla"""
        if not self.history:
            return "Henüz test verisi yok."

        # Model bazında istatistikler
        model_stats = {}
        for model_name in self.models.keys():
            correct = 0
            total = 0

            class_stats = {class_name: {'correct': 0, 'total': 0} for class_name in self.class_names}

            for record in self.history:
                if model_name in record['predictions']:
                    total += 1
                    true_label = record['true_label']
                    predicted = record['predictions'][model_name]['predicted_class']

                    class_stats[true_label]['total'] += 1

                    if predicted == true_label:
                        correct += 1
                        class_stats[true_label]['correct'] += 1

            if total > 0:
                accuracy = (correct / total) * 100
                model_stats[model_name] = {
                    'accuracy': accuracy,
                    'correct': correct,
                    'total': total,
                    'class_stats': class_stats
                }

        # İstatistikleri formatla
        stats_text = f"📈 **PERFORMANS İSTATİSTİKLERİ** (Son {len(self.history)} test)\n\n"

        # Genel başarı oranları
        sorted_models = sorted(model_stats.items(), key=lambda x: x[1]['accuracy'], reverse=True)

        stats_text += "🏆 **GENEL BAŞARI ORANLARI:**\n"
        for i, (model_name, stats) in enumerate(sorted_models, 1):
            stats_text += f"{i}. **{model_name}**: {stats['accuracy']:.1f}% ({stats['correct']}/{stats['total']})\n"

        stats_text += "\n📋 **SINIF BAZINDA PERFORMANS:**\n"

        # Sınıf bazında en iyi modeller
        for class_name in self.class_names:
            best_acc = 0
            best_model = ""

            for model_name, stats in model_stats.items():
                class_total = stats['class_stats'][class_name]['total']
                if class_total > 0:
                    class_acc = (stats['class_stats'][class_name]['correct'] / class_total) * 100
                    if class_acc > best_acc:
                        best_acc = class_acc
                        best_model = model_name

            if best_model:
                stats_text += f"• **{class_name}**: {best_model} ({best_acc:.1f}%)\n"

        return stats_text

    def get_detailed_history(self):
        """Detaylı geçmiş tablosunu oluştur"""
        if not self.history:
            return pd.DataFrame()

        data = []
        for i, record in enumerate(self.history, 1):
            row = {
                'Test #': i,
                'Tarih': record['timestamp'][:19],  # Sadece tarih-saat kısmı
                'Gerçek': record['true_label']
            }

            # Her model için tahmin ve doğruluk ekle
            for model_name in self.models.keys():
                if model_name in record['predictions']:
                    predicted = record['predictions'][model_name]['predicted_class']
                    confidence = record['predictions'][model_name]['confidence']
                    is_correct = "✅" if predicted == record['true_label'] else "❌"

                    row[f'{model_name}_Tahmin'] = predicted
                    row[f'{model_name}_Güven'] = f"{confidence:.1f}%"
                    row[f'{model_name}_Doğru'] = is_correct

            data.append(row)

        return pd.DataFrame(data)

# Global model tester instance
model_tester = ModelTester()

def predict_interface(image):
    """Gradio arayüzü için tahmin fonksiyonu"""
    return model_tester.predict_all_models(image)

def submit_interface(true_label):
    """Gradio arayüzü için doğrulama fonksiyonu"""
    return model_tester.submit_ground_truth(true_label)

def get_stats_interface():
    """İstatistikleri getir"""
    return model_tester.get_accuracy_stats()

def get_history_interface():
    """Geçmişi getir"""
    return model_tester.get_detailed_history()

# Gradio arayüzü
def create_interface():
    with gr.Blocks(title="Gürültü Sınıflandırma Modeli Test Sistemi", theme=gr.themes.Soft()) as iface:
        gr.Markdown("""
        # 🔬 Gürültü Sınıflandırma Modeli Test Sistemi

        Bu arayüz 5 farklı model ile gürültü tiplerini sınıflandırır ve performanslarını karşılaştırır.

        **Kullanım:**
        1. Bir resim yükleyin
        2. Modellerin tahminlerini görün
        3. Gerçek gürültü tipini seçin
        4. Sonuçları değerlendirin
        """)

        with gr.Row():
            with gr.Column(scale=1):
                # Resim yükleme
                image_input = gr.Image(
                    type="pil",
                    label="📷 Test Resmi Yükleyin",
                    height=300
                )

                # Tahmin butonu
                predict_btn = gr.Button("🔮 Tahmin Yap", variant="primary")

                # Gerçek etiket seçimi
                true_label = gr.Dropdown(
                    choices=model_tester.class_names,
                    label="✅ Gerçek Gürültü Tipi",
                    info="Resimin gerçek gürültü tipini seçin"
                )

                # Doğrulama butonu
                submit_btn = gr.Button("📝 Doğrulamayı Kaydet", variant="secondary")

            with gr.Column(scale=2):
                # Tahmin sonuçları
                predictions_output = gr.Markdown(
                    label="🤖 Model Tahminleri",
                    value="Model tahminleri burada görünecek..."
                )

                # Doğrulama sonuçları
                validation_output = gr.Markdown(
                    label="📊 Doğrulama Sonuçları",
                    value="Doğrulama sonuçları burada görünecek..."
                )

        with gr.Row():
            with gr.Column():
                # İstatistikler
                stats_output = gr.Markdown(
                    label="📈 Performans İstatistikleri",
                    value=model_tester.get_accuracy_stats()
                )

                # İstatistikleri güncelleme butonu
                refresh_stats_btn = gr.Button("🔄 İstatistikleri Güncelle")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 📋 Test Geçmişi")
                history_output = gr.Dataframe(
                    value=model_tester.get_detailed_history(),
                    label="Detaylı Test Geçmişi",
                    interactive=False
                )

                refresh_history_btn = gr.Button("🔄 Geçmişi Güncelle")

        # Event handlers
        predict_btn.click(
            fn=predict_interface,
            inputs=[image_input],
            outputs=[predictions_output, stats_output, validation_output]
        )

        submit_btn.click(
            fn=submit_interface,
            inputs=[true_label],
            outputs=[validation_output, stats_output]
        )

        refresh_stats_btn.click(
            fn=get_stats_interface,
            outputs=[stats_output]
        )

        refresh_history_btn.click(
            fn=get_history_interface,
            outputs=[history_output]
        )

        # Otomatik güncellemeler
        submit_btn.click(
            fn=get_history_interface,
            outputs=[history_output]
        )

    return iface

if __name__ == "__main__":
    print("🚀 Gradio arayüzü başlatılıyor...")
    print(f"📱 Cihaz: {model_tester.device}")
    print(f"🔢 Yüklenen model sayısı: {len(model_tester.models)}")

    # Arayüzü oluştur ve başlat
    interface = create_interface()
    interface.launch(
        share=False,  # Public link oluştur
        server_name="0.0.0.0",  # Tüm IP'lerden erişim
        server_port=7860,  # Port
        show_error=True
    )